In [1]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime
import time

In [2]:
import os
from dotenv import load_dotenv 
load_dotenv()

True

In [3]:
load_dotenv()
tmdb_key = os.getenv("tmdb_key")

In [4]:
load_dotenv()
tmdb_access_token = os.getenv("tmdb_access_token")

In [5]:
bechdel_new_df = pd.read_csv('../data/Bechdel_updated.csv')
bechdel_new_df

,title,year,bt_score,dubious,imdbid,id,submitterid,date,visible
0,Gemini Man,2019.0,1.0,0.0,1025100.0,8868.0,17383.0,2019-10-12 10:46:05,1.0
1,Ford v Ferrari,2019.0,0.0,0.0,1950186.0,8869.0,17384.0,2019-10-12 10:48:31,1.0
2,Hustlers,2019.0,3.0,0.0,5503686.0,8870.0,17389.0,2019-10-14 13:49:26,1.0
3,Maleficent: Mistress of Evil,2019.0,3.0,0.0,4777008.0,8876.0,17403.0,2019-10-26 22:39:21,1.0
4,"Laundromat, The",2019.0,3.0,0.0,5865326.0,8877.0,17405.0,2019-10-27 07:11:37,1.0
...,...,...,...,...,...,...,...,...,...
760,Winnie-the-Pooh: Blood and Honey,2023.0,3.0,0.0,19623240.0,10995.0,21441.0,2023-09-03 11:21:01,1.0
761,Haunted Mansion,2023.0,3.0,0.0,1695843.0,10996.0,21442.0,2023-09-03 18:32:34,1.0
762,"Portable Door, The",2023.0,3.0,0.0,11820950.0,11009.0,21466.0,2023-09-16 03:43:53,1.0
763,Anatomie d&#39;une chute,2023.0,3.0,0.0,17009710.0,11010.0,21469.0,2023-09-19 08:14:23,1.0


In [6]:
# let's check with values i should get from TMDB
movies = pd.read_csv("../data/movies.csv",index_col=0)
movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7279 entries, 0 to 7634
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   title                       7279 non-null   object 
 1   year                        7279 non-null   int64  
 2   bt_score                    7279 non-null   int64  
 3   dubious                     7279 non-null   int64  
 4   imdbid                      7279 non-null   int64  
 5   tmdbId                      7279 non-null   int64  
 6   genres                      7279 non-null   object 
 7   popularity                  7279 non-null   float64
 8   production_companies        7279 non-null   object 
 9   production_countries        7279 non-null   object 
 10  release_date                7279 non-null   object 
 11  revenue                     7279 non-null   float64
 12  vote_average                7279 non-null   float64
 13  vote_count                  7279 non-n

In [ ]:
# tmbdid, genres, genres, popularity, production_companies, production_countries, release_date, revenue, vote_average  
# vote_count, crew, budget, cast_gender, crew_gender, cast_female_representation, crew_female_representation 

In [ ]:
bechdel_id = list(bechdel_new_df["imdbid"])
movies_id = list(movies["imdbid"])
movies_id = [el for el in bechdel_id if el not in movies_id]
len(movies_id)

In [ ]:
#movies_id

In [ ]:
# change imdbit in the format ddddddd
def get_id(dep_id):
    try:
        dep_id = str(int(dep_id))
        l = len(dep_id)
        new_id = '0'*(7-l)+dep_id
        return new_id
    except:
        pass
    
def change_all_id(lista):
    for i in range(len(lista)):
        lista[i] = get_id(lista[i])
    return lista

In [ ]:
movies_id = change_all_id(movies_id)

In [ ]:
#movies_id

In [ ]:
headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {tmdb_access_token}"
}

In [ ]:
credit_url = f"https://api.themoviedb.org/3/movie/tt9059704/credits?language=en-US"
credit_response = requests.get(credit_url, headers=headers).json()
credit_response

In [ ]:
cast = credit_response['cast']
crew = credit_response['crew']

In [ ]:
details_url = f"https://api.themoviedb.org/3/movie/tt12198524?language=en-US"
response = requests.get(details_url, headers=headers).json()
response

In [ ]:
genres = [el["name"] for el in response["genres"]]
imdb_id = response["imdb_id"].replace('tt','')
budget = response["budget"]
popularity = response["popularity"]
production_companies = response['production_companies']
release_date = response['release_date']
revenue = response['revenue']
vote_average = response['vote_average']
vote_count = response['vote_count']

In [ ]:
def get_movie_info(imdbid_list):
    headers = {
        "accept": "application/json",
        "Authorization": f"Bearer {tmdb_access_token}"
    }
    
    new_movie_list = []
    cont=0
    
    for imbdid in imdbid_list:
        movie_dict={}
        time.sleep(1)
        details_url = f"https://api.themoviedb.org/3/movie/tt{imbdid}?language=en-US"        
        credits_url = f"https://api.themoviedb.org/3/movie/tt{imbdid}/credits?language=en-US"
        
        
        details_response = requests.get(details_url, headers=headers).json()
        try: 
            movie_dict["imdbid"] = imbdid
            movie_dict["genres"] = [el["name"] for el in details_response["genres"]]           
            movie_dict["budget"] = details_response["budget"]
            movie_dict["popularity"] = details_response["popularity"]
            movie_dict["production_companies"] = details_response['production_companies']
            movie_dict["release_date"] = details_response['release_date']
            movie_dict["revenue"] = details_response['revenue']
            movie_dict["vote_average"] = details_response['vote_average']
            movie_dict["vote_count"] = details_response['vote_count']
        except:
            pass
    
        credit_response = requests.get(credits_url, headers=headers).json()
        
        try:
            movie_dict['cast'] = credit_response['cast']
            movie_dict['crew'] = credit_response['crew']
        except:
            pass
        
        new_movie_list.append(movie_dict)
        
        if cont%20==0:
            pd.DataFrame(new_movie_list).to_csv("../data/checkpoint_df.csv",index=False)
        cont+=1
        
    return new_movie_list

In [ ]:
movie_list = get_movie_info(movies_id)

In [6]:
df = pd.read_csv("../data/checkpoint_df.csv")
df

,imdbid,genres,budget,popularity,production_companies,release_date,revenue,vote_average,vote_count,cast,crew
0,1025100.0,"['Science Fiction', 'Action', 'Thriller']",140000000.0,40.054,"[{'id': 82819, 'logo_path': '/gXfFl9pRPaoaq14j...",2019-10-02,173469516.0,6.313,4778.0,"[{'adult': False, 'gender': 2, 'id': 2888, 'kn...","[{'adult': False, 'gender': 1, 'id': 2952, 'kn..."
1,1950186.0,"['Drama', 'Action', 'History']",97600000.0,64.900,"[{'id': 25, 'logo_path': '/qZCc1lty5FzX30aOCVR...",2019-11-13,225508210.0,8.012,7078.0,"[{'adult': False, 'gender': 2, 'id': 3894, 'kn...","[{'adult': False, 'gender': 1, 'id': 1720, 'kn..."
2,5503686.0,"['Drama', 'Comedy', 'Crime']",20000000.0,33.157,"[{'id': 13184, 'logo_path': '/pfUB1a62jSMIqp4X...",2019-09-12,157600000.0,6.487,2773.0,"[{'adult': False, 'gender': 1, 'id': 206444, '...","[{'adult': False, 'gender': 1, 'id': 3429, 'kn..."
3,4777008.0,"['Family', 'Fantasy', 'Adventure', 'Action']",185000000.0,46.907,"[{'id': 2, 'logo_path': '/wdrCwmRnLFJhEoH8GSfy...",2019-10-16,491730089.0,7.321,5749.0,"[{'adult': False, 'gender': 1, 'id': 11701, 'k...","[{'adult': False, 'gender': 2, 'id': 1722, 'kn..."
4,5865326.0,"['Drama', 'Crime', 'Comedy']",0.0,15.499,"[{'id': 10039, 'logo_path': '/b8w4LldQolMKiLZw...",2019-09-27,0.0,6.004,1120.0,"[{'adult': False, 'gender': 1, 'id': 5064, 'kn...","[{'adult': False, 'gender': 2, 'id': 1884, 'kn..."
...,...,...,...,...,...,...,...,...,...,...,...
636,14309446.0,['Comedy'],0.0,32.523,"[{'id': 160027, 'logo_path': None, 'name': 'Pa...",2022-08-26,0.0,5.927,770.0,"[{'adult': False, 'gender': 2, 'id': 55638, 'k...","[{'adult': False, 'gender': 1, 'id': 17150, 'k..."
637,13139228.0,"['Drama', 'Romance']",0.0,18.508,"[{'id': 20580, 'logo_path': '/oRR9EXVoKP9szDkV...",2022-10-20,0.0,7.764,592.0,"[{'adult': False, 'gender': 2, 'id': 1052109, ...","[{'adult': False, 'gender': 1, 'id': 471, 'kno..."
638,14641788.0,"['Adventure', 'Mystery', 'Crime']",0.0,37.480,"[{'id': 923, 'logo_path': '/5UQsZrfbfG2dYJbx8D...",2022-11-04,0.0,7.536,2206.0,"[{'adult': False, 'gender': 1, 'id': 1356210, ...","[{'adult': False, 'gender': 2, 'id': 3079, 'kn..."
639,15218000.0,"['Comedy', 'Romance']",10000000.0,14.970,"[{'id': 24021, 'logo_path': '/urq2p5ColCf657b5...",2022-06-09,0.0,7.162,195.0,"[{'adult': False, 'gender': 2, 'id': 2371545, ...","[{'adult': False, 'gender': 2, 'id': 19759, 'k..."


In [7]:
#df = pd.DataFrame(movie_list)
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)
df.reset_index(drop=True,inplace=True)

In [8]:
df

,imdbid,genres,budget,popularity,production_companies,release_date,revenue,vote_average,vote_count,cast,crew
0,1025100.0,"['Science Fiction', 'Action', 'Thriller']",140000000.0,40.054,"[{'id': 82819, 'logo_path': '/gXfFl9pRPaoaq14j...",2019-10-02,173469516.0,6.313,4778.0,"[{'adult': False, 'gender': 2, 'id': 2888, 'kn...","[{'adult': False, 'gender': 1, 'id': 2952, 'kn..."
1,1950186.0,"['Drama', 'Action', 'History']",97600000.0,64.900,"[{'id': 25, 'logo_path': '/qZCc1lty5FzX30aOCVR...",2019-11-13,225508210.0,8.012,7078.0,"[{'adult': False, 'gender': 2, 'id': 3894, 'kn...","[{'adult': False, 'gender': 1, 'id': 1720, 'kn..."
2,5503686.0,"['Drama', 'Comedy', 'Crime']",20000000.0,33.157,"[{'id': 13184, 'logo_path': '/pfUB1a62jSMIqp4X...",2019-09-12,157600000.0,6.487,2773.0,"[{'adult': False, 'gender': 1, 'id': 206444, '...","[{'adult': False, 'gender': 1, 'id': 3429, 'kn..."
3,4777008.0,"['Family', 'Fantasy', 'Adventure', 'Action']",185000000.0,46.907,"[{'id': 2, 'logo_path': '/wdrCwmRnLFJhEoH8GSfy...",2019-10-16,491730089.0,7.321,5749.0,"[{'adult': False, 'gender': 1, 'id': 11701, 'k...","[{'adult': False, 'gender': 2, 'id': 1722, 'kn..."
4,5865326.0,"['Drama', 'Crime', 'Comedy']",0.0,15.499,"[{'id': 10039, 'logo_path': '/b8w4LldQolMKiLZw...",2019-09-27,0.0,6.004,1120.0,"[{'adult': False, 'gender': 1, 'id': 5064, 'kn...","[{'adult': False, 'gender': 2, 'id': 1884, 'kn..."
...,...,...,...,...,...,...,...,...,...,...,...
609,14309446.0,['Comedy'],0.0,32.523,"[{'id': 160027, 'logo_path': None, 'name': 'Pa...",2022-08-26,0.0,5.927,770.0,"[{'adult': False, 'gender': 2, 'id': 55638, 'k...","[{'adult': False, 'gender': 1, 'id': 17150, 'k..."
610,13139228.0,"['Drama', 'Romance']",0.0,18.508,"[{'id': 20580, 'logo_path': '/oRR9EXVoKP9szDkV...",2022-10-20,0.0,7.764,592.0,"[{'adult': False, 'gender': 2, 'id': 1052109, ...","[{'adult': False, 'gender': 1, 'id': 471, 'kno..."
611,14641788.0,"['Adventure', 'Mystery', 'Crime']",0.0,37.480,"[{'id': 923, 'logo_path': '/5UQsZrfbfG2dYJbx8D...",2022-11-04,0.0,7.536,2206.0,"[{'adult': False, 'gender': 1, 'id': 1356210, ...","[{'adult': False, 'gender': 2, 'id': 3079, 'kn..."
612,15218000.0,"['Comedy', 'Romance']",10000000.0,14.970,"[{'id': 24021, 'logo_path': '/urq2p5ColCf657b5...",2022-06-09,0.0,7.162,195.0,"[{'adult': False, 'gender': 2, 'id': 2371545, ...","[{'adult': False, 'gender': 2, 'id': 19759, 'k..."


In [9]:
# junta los 3 dataframes y elimina columnas duplicadas
# bechdel_new_df = pd.read_csv("../data/Bechdel_updated.csv")

new_movies_df = pd.merge(bechdel_new_df, df, left_on='imdbid', right_on='imdbid', how='inner')

new_movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   title                 614 non-null    object 
 1   year                  614 non-null    float64
 2   bt_score              614 non-null    float64
 3   dubious               614 non-null    float64
 4   imdbid                614 non-null    float64
 5   id                    614 non-null    float64
 6   submitterid           614 non-null    float64
 7   date                  614 non-null    object 
 8   visible               614 non-null    float64
 9   genres                614 non-null    object 
 10  budget                614 non-null    float64
 11  popularity            614 non-null    float64
 12  production_companies  614 non-null    object 
 13  release_date          614 non-null    object 
 14  revenue               614 non-null    float64
 15  vote_average          6

In [10]:
new_movies_df.drop(columns=["id","submitterid","date","visible"], inplace=True)

In [11]:
new_movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   title                 614 non-null    object 
 1   year                  614 non-null    float64
 2   bt_score              614 non-null    float64
 3   dubious               614 non-null    float64
 4   imdbid                614 non-null    float64
 5   genres                614 non-null    object 
 6   budget                614 non-null    float64
 7   popularity            614 non-null    float64
 8   production_companies  614 non-null    object 
 9   release_date          614 non-null    object 
 10  revenue               614 non-null    float64
 11  vote_average          614 non-null    float64
 12  vote_count            614 non-null    float64
 13  cast                  614 non-null    object 
 14  crew                  614 non-null    object 
dtypes: float64(9), object(6

In [48]:
# function to get cast/crew gender

In [14]:
for index, movie in new_movies_df.iterrows():
    companies = eval(movie['production_companies'])
    companies_list = []

    for company in companies:
        company_name = company['name']
        companies_list.append(company_name)

    new_movies_df.at[index, 'production_companies'] = companies_list

In [15]:
for index, movie in new_movies_df.iterrows():
    date = datetime.strptime(movie['release_date'], '%Y-%m-%d')
    reformatted_date = date.strftime('%d/%m/%Y')

    new_movies_df.at[index, 'release_date'] = reformatted_date

In [16]:
new_movies_df['cast_gender'] = pd.NA


for index, movie in new_movies_df.iterrows():
    cast = eval(movie['cast'])
    gender_list = []

    for cast_member in cast:
        gender = cast_member['gender']
        gender_list.append(gender)

    new_movies_df.at[index, 'cast_gender'] = gender_list

In [17]:
new_movies_df['crew_gender'] = pd.NA


for index, movie in new_movies_df.iterrows():
    crew = eval(movie['crew'])
    gender_list = []

    for crew_member in crew:
        gender = crew_member['gender']
        gender_list.append(gender)

    new_movies_df.at[index, 'crew_gender'] = gender_list

In [18]:
new_movies_df['cast_female_representation'] = pd.NA
new_movies_df['crew_female_representation'] = pd.NA


for index, movie in new_movies_df.iterrows():
    if len(movie['cast_gender']) != 0:
        new_movies_df.at[index, 'cast_female_representation'] = 100*(movie['cast_gender'].count(1)/len(movie['cast_gender']))

    if len(movie['crew_gender']) != 0:
        new_movies_df.at[index, 'crew_female_representation'] = 100*(movie['crew_gender'].count(1)/len(movie['crew_gender']))


new_movies_df = new_movies_df.dropna().copy()
new_movies_df['cast_female_representation'] = new_movies_df['cast_female_representation'].astype('float64')
new_movies_df['crew_female_representation'] = new_movies_df['crew_female_representation'].astype('float64')

In [12]:
movies = pd.read_csv("../data/movies.csv", index_col=0)
movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7279 entries, 0 to 7634
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   title                       7279 non-null   object 
 1   year                        7279 non-null   int64  
 2   bt_score                    7279 non-null   int64  
 3   dubious                     7279 non-null   int64  
 4   imdbid                      7279 non-null   int64  
 5   tmdbId                      7279 non-null   int64  
 6   genres                      7279 non-null   object 
 7   popularity                  7279 non-null   float64
 8   production_companies        7279 non-null   object 
 9   production_countries        7279 non-null   object 
 10  release_date                7279 non-null   object 
 11  revenue                     7279 non-null   float64
 12  vote_average                7279 non-null   float64
 13  vote_count                  7279 non-n

In [57]:
new_movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 613 entries, 0 to 613
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   title                       613 non-null    object 
 1   year                        613 non-null    float64
 2   bt_score                    613 non-null    float64
 3   dubious                     613 non-null    float64
 4   imdbid                      613 non-null    float64
 5   genres                      613 non-null    object 
 6   budget                      613 non-null    float64
 7   popularity                  613 non-null    float64
 8   production_companies        613 non-null    object 
 9   release_date                613 non-null    object 
 10  revenue                     613 non-null    float64
 11  vote_average                613 non-null    float64
 12  vote_count                  613 non-null    float64
 13  cast                        613 non-null

In [13]:
all_movies = pd.concat([movies,new_movies_df], axis=0)
all_movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7893 entries, 0 to 613
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   title                       7893 non-null   object 
 1   year                        7893 non-null   float64
 2   bt_score                    7893 non-null   float64
 3   dubious                     7893 non-null   float64
 4   imdbid                      7893 non-null   float64
 5   tmdbId                      7279 non-null   float64
 6   genres                      7893 non-null   object 
 7   popularity                  7893 non-null   float64
 8   production_companies        7893 non-null   object 
 9   production_countries        7279 non-null   object 
 10  release_date                7893 non-null   object 
 11  revenue                     7893 non-null   float64
 12  vote_average                7893 non-null   float64
 13  vote_count                  7893 non-nu

In [60]:
all_movies.dropna(how='all',inplace=True)

In [59]:
all_movies.drop(columns=["cast","crew"],inplace=True)

In [61]:
all_movies.drop_duplicates(subset="imdbid", inplace=True)

In [62]:
all_movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7884 entries, 0 to 613
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   title                       7884 non-null   object 
 1   year                        7884 non-null   float64
 2   bt_score                    7884 non-null   float64
 3   dubious                     7884 non-null   float64
 4   imdbid                      7884 non-null   float64
 5   tmdbId                      7271 non-null   float64
 6   genres                      7884 non-null   object 
 7   popularity                  7884 non-null   float64
 8   production_companies        7884 non-null   object 
 9   production_countries        7271 non-null   object 
 10  release_date                7884 non-null   object 
 11  revenue                     7884 non-null   float64
 12  vote_average                7884 non-null   float64
 13  vote_count                  7884 non-nu

In [64]:
all_movies.reset_index(drop=True, inplace=True)

In [65]:
all_movies

,title,year,bt_score,dubious,imdbid,tmdbId,genres,popularity,production_companies,production_countries,release_date,revenue,vote_average,vote_count,budget,cast_gender,crew_gender,cast_female_representation,crew_female_representation
0,Cinderella,1899.0,3.0,0.0,230.0,114108.0,"['Drama', 'Family', 'Fantasy', 'Romance']",3.762,"['Star-Film', 'Georges Méliès']",['France'],01/10/1899,0.0,6.100,79.0,0.0,"[2, 0, 1, 0, 1, 0]","[2, 2, 2, 2]",33.333333,0.000000
1,Gretchen the Greenhorn,1916.0,3.0,0.0,6745.0,126925.0,[],1.942,[],[],03/09/1916,0.0,6.100,7.0,0.0,"[1, 2, 2, 2]","[2, 2, 2]",25.000000,0.000000
2,Snow White,1916.0,3.0,0.0,7361.0,174598.0,"['Fantasy', 'Drama']",0.817,[],[],24/12/1916,0.0,5.700,9.0,0.0,"[0, 1, 2, 2, 0]",[2],20.000000,0.000000
3,The Poor Little Rich Girl,1917.0,3.0,0.0,8443.0,95866.0,"['Romance', 'Comedy', 'Drama', 'Fantasy']",2.023,['Artcraft Pictures Corporation'],['United States of America'],05/03/1917,0.0,5.800,23.0,0.0,"[1, 0, 0, 0, 2, 0, 1, 1, 0, 2, 0, 1, 1]","[2, 2, 1, 1, 2, 2, 0]",38.461538,28.571429
4,Stella Maris,1918.0,3.0,0.0,9652.0,70753.0,['Drama'],1.310,['Mary Pickford Company'],['United States of America'],21/01/1918,2400000.0,6.900,19.0,0.0,"[1, 1, 2, 2, 1, 0, 0, 2]","[1, 2, 2, 0, 2, 2]",37.500000,16.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7879,Me Time,2022.0,1.0,0.0,14309446.0,NaN,['Comedy'],32.523,"[Particular Pictures, HartBeat Productions]",NaN,26/08/2022,0.0,5.927,770.0,0.0,"[2, 2, 1, 2, 1, 2, 2, 2, 2, 1, 1, 2, 1, 1, 2, ...","[1, 2, 2, 2, 1, 2, 2, 2, 1, 0, 2, 1, 1, 2, 0, ...",25.806452,29.411765
7880,My Policeman,2022.0,2.0,0.0,13139228.0,NaN,"['Drama', 'Romance']",18.508,"[Amazon Studios, Berlanti Productions, Indepen...",NaN,20/10/2022,0.0,7.764,592.0,0.0,"[2, 3, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 0, ...","[1, 2, 2, 0, 2, 1, 0, 2, 2, 0, 0, 2, 1, 2, 2, ...",17.391304,9.375000
7881,Enola Holmes 2,2022.0,3.0,0.0,14641788.0,NaN,"['Adventure', 'Mystery', 'Crime']",37.480,"[Legendary Pictures, PCMA Productions]",NaN,04/11/2022,0.0,7.536,2206.0,0.0,"[1, 2, 2, 2, 1, 2, 1, 1, 1, 0, 2, 1, 1, 0, 2, ...","[2, 1, 1, 2, 0, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, ...",14.285714,16.216216
7882,Fire Island,2022.0,3.0,0.0,15218000.0,NaN,"['Comedy', 'Romance']",14.970,"[Jax Media, Searchlight Pictures]",NaN,09/06/2022,0.0,7.162,195.0,10000000.0,"[2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, ...","[2, 1, 2, 2, 0, 0, 2, 2, 2, 0, 2, 1, 2, 0, 0, ...",10.714286,10.937500


In [67]:
all_movies.to_csv("../data/all_movies.csv")